In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json
import os

In [8]:
dataset_dir = 'PlantVillage'

In [9]:
img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation'
)

Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.


In [10]:
labels = {v: k for k, v in train_data.class_indices.items()}

os.makedirs('model', exist_ok=True)
with open('model/labels.json', 'w') as f:
    json.dump(labels, f)

In [11]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(len(train_data.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,306,575 (12.61 MB)

 Trainable params: 3,306,575 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15
)

Epoch 1/15
517/517 ━━━━━━━━━━━━━━━━━━━━ 233s 447ms/step - accuracy: 0.3499 - loss: 2.0093 - val_accuracy: 0.7567 - val_loss: 0.7744
Epoch 2/15
517/517 ━━━━━━━━━━━━━━━━━━━━ 123s 238ms/step - accuracy: 0.6985 - loss: 0.9023 - val_accuracy: 0.8030 - val_loss: 0.5879
Epoch 3/15
517/517 ━━━━━━━━━━━━━━━━━━━━ 109s 211ms/step - accuracy: 0.7752 - loss: 0.6714 - val_accuracy: 0.8498 - val_loss: 0.4472
Epoch 4/15
517/517 ━━━━━━━━━━━━━━━━━━━━ 111s 214ms/step - accuracy: 0.8263 - loss: 0.5148 - val_accuracy: 0.8721 - val_loss: 0.3617
Epoch 5/15
517/517 ━━━━━━━━━━━━━━━━━━━━ 121s 235ms/step - accuracy: 0.8589 - loss: 0.4133 - val_accuracy: 0.8763 - val_loss: 0.3757
Epoch 6/15
517/517 ━━━━━━━━━━━━━━━━━━━━ 116s 225ms/step - accuracy: 0.8812 - loss: 0.3537 - val_accuracy: 0.8935 - val_loss: 0.3249
Epoch 7/15
517/517 ━━━━━━━━━━━━━━━━━━━━ 119s 230ms/step - accuracy: 0.9035 - loss: 0.2830 - val_accuracy: 0.9064 - val_loss: 0.2947
Epoch 8/15
517/517 ━━━━━━━━━━━━━━━━━━━━ 119s 229ms/step - accuracy: 0.9132 -

In [13]:
model.save('model/plant_disease_model.h5')
print("✅ Model training complete and saved to 'model/plant_disease_model.h5'")

✅ Model training complete and saved to 'model/plant_disease_model.h5'
